In [1]:
# importing the necessary libraries
import pandas as pd
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

import undetected_chromedriver as uc

import time

In [2]:
#importing the given review dataset
reviews = pd.read_csv(r'technicalAssessment-GoogleReviews.csv')
reviews.head()

,name,storeID,fullAddress,phone,website,overallRating,numberReviews,reviewer,reviewRating,reviewText,reviewDate
0,la Madeleine,6206,"6109 Berkshire Ln Ste A, Dallas, TX 75225",(945) 279-0912,https://lamadeleine.com/locations/berkshire,3.1,12,Olga Garcia,2,How do you have 3 people working a lunch shift...,2024-03-05T19:11:20.386Z
1,la Madeleine,6206,"6109 Berkshire Ln Ste A, Dallas, TX 75225",(945) 279-0912,https://lamadeleine.com/locations/berkshire,3.1,12,Melissa Trevino,1,Just be aware of being double charged during t...,2024-03-05T19:11:20.386Z
2,la Madeleine,6206,"6109 Berkshire Ln Ste A, Dallas, TX 75225",(945) 279-0912,https://lamadeleine.com/locations/berkshire,3.1,12,Roger Stierman,5,"Always delicious,new crew being trainedFood: 5...",2024-02-27T19:11:20.386Z
3,la Madeleine,6206,"6109 Berkshire Ln Ste A, Dallas, TX 75225",(945) 279-0912,https://lamadeleine.com/locations/berkshire,3.1,12,Eric Brown,5,Very clean and nice location with friendly peo...,2024-02-27T19:11:20.387Z
4,la Madeleine,6206,"6109 Berkshire Ln Ste A, Dallas, TX 75225",(945) 279-0912,https://lamadeleine.com/locations/berkshire,3.1,12,AJ D.,2,It’s not even busy so I can’t BELIEVE how long...,2024-02-19T00:00:00.000Z


In [3]:
reviews.columns

Index(['name', 'storeID', 'fullAddress', 'phone', 'website', 'overallRating',
       'numberReviews', 'reviewer', 'reviewRating', 'reviewText',
       'reviewDate'],
      dtype='object')

In [4]:
# checking the data for nul values
reviews.isnull().sum()

name                 0
storeID              0
fullAddress          0
phone                0
website            202
overallRating        0
numberReviews        0
reviewer             0
reviewRating         0
reviewText       25131
reviewDate           0
dtype: int64

In [5]:
for col in reviews:
    print(col + ': ' +str(len(reviews[col].unique())))

name: 8
storeID: 90
fullAddress: 86
phone: 86
website: 81
overallRating: 12
numberReviews: 87
reviewer: 57444
reviewRating: 5
reviewText: 36224
reviewDate: 1076


## Step 1: Extract location data

### Scraping the data

* Upon inspecting the site, the website provided for locations has details like name of the location, phone number, website for the restaurant, which also has the storeID.
* The storeID for the restaurant is a possible key value that could be used to match the locations and the reviews.

In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.page_load_strategy = 'normal'
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("start-maximized")

driver = uc.Chrome(options=chrome_options)

url = 'https://lamadeleine.com/locations'
driver.get(url)

could not detect version_main.therefore, we are assuming it is chrome 108 or higher


In [7]:
map_element = driver.find_element(By.CLASS_NAME, "map__container")
map_element.click()
time.sleep(10)
actions = ActionChains(driver)

#zoom out the map to ensure all the US locations are in view 
for _ in range(10):
    actions.send_keys(Keys.SUBTRACT).perform()
    
#parse the content of the html source
time.sleep(10)
all_buttons = driver.find_elements(By.CLASS_NAME, "location__content")

In [8]:
len(all_buttons)

91

In [9]:
locationName = []
phoneNumber = []
locURL = []

for i in range(len(all_buttons)):
    print(i)
    map_element = driver.find_element(By.CLASS_NAME, "map__container")
    map_element.click()
    time.sleep(10)
    actions = ActionChains(driver)

    #zoom out the map to ensure all the US locations are in view 
    for _ in range(10):
        actions.send_keys(Keys.SUBTRACT).perform()

    #parse the content of the html source
    time.sleep(10)
    all_buttons = driver.find_elements(By.CLASS_NAME, "location__content")
    button = all_buttons[i]
    
    # get location Name
    locationName.append(button.find_element(By.CLASS_NAME, "location__name").text)
    
    # get phone Number
    phoneNumber.append(button.find_element(By.CLASS_NAME, "location__phone").text)
    
    # get location buttons for address and store ID
    locationButton = button.find_element(By.CLASS_NAME, "location__link")
    locationButton.click()
    time.sleep(5)
    locURL.append(driver.current_url)
    driver.back()
    time.sleep(5)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90


In [10]:
print(len(locationName))
print(len(phoneNumber))
print(len(locURL))

91
91
91


In [11]:
chrome_options = webdriver.ChromeOptions()
chrome_options.page_load_strategy = 'normal'
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("start-maximized")

# Setup WebDriver
driver2 = uc.Chrome(chrome_options)

storeID = []
ln1 = []
ln2 = []
ln3 = []

for loc in locURL:
    driver2.get(loc)
    driver2.refresh()
    time.sleep(5)
    soup2 = BeautifulSoup(driver2.page_source, "html.parser")
    results2 = soup2.find("body")
    
    #get store ID
    storeID.append(results2.get("class")[-1])
    
    #get address elements
    addr1 = results2.find("div", class_="address__line__1")
    addr2 = results2.find("div", class_="address__line__2")
    addr3 = results2.find("div", class_="address__line__3")
    
    ln1.append(addr1.get_text() if addr1 else None)
    ln2.append(addr2.get_text() if addr2 else None)
    ln3.append(addr3.get_text() if addr3 else None)

could not detect version_main.therefore, we are assuming it is chrome 108 or higher


In [12]:
print(len(storeID))
print(len(ln1))
print(len(ln2))
print(len(ln3))

91
91
91
91


The fields used to scrape the data have been obtained after "inspecting" the website

In [13]:
# creating a data frame with the scraped values
web_data = pd.DataFrame(zip(locationName, phoneNumber, storeID, ln1, ln2, ln3),\
                        columns=['locationName','phoneNumber','storeID', 'addressLine1','addressLine2','addressLine3'])
web_data.head()

,locationName,phoneNumber,storeID,addressLine1,addressLine2,addressLine3
0,San Jacinto,214-220-3911,postid-1733,2100 Ross Avenue,Suite 120,"Dallas, TX 75201"
1,Park District - Petite Market & Bakery,972-913-2280,postid-4691,2100 Olive Street,Suite 150,"Dallas, TX 75201"
2,Dallas Love Field,214-358-7614,postid-1685,8008 Cedar Springs Rd.,Terminal 2 Gate 6,"Dallas, TX 75235"
3,Mockingbird,214-696-0800,postid-1713,3072 Mockingbird Lane,None,"Dallas, TX 75205"
4,Berkshire,945-279-0912,postid-6206,6109 Berkshire Ln,Ste A,"Dallas, Texas 75225"


I have noticed that there are a few missing values form the second address line component in the web pages. So, here I check if any other column has missing values. In case it does, the code will scrape the values from those pags again

In [14]:
missing_val_columns = web_data.columns[web_data.isnull().any().values].tolist()

if len(missing_val_columns)>1:
    missing_vals = web_data[web_data.isnull().any(axis=1)].index.tolist()

In [15]:
if missing_vals != None:
    chrome_options = webdriver.ChromeOptions()
    chrome_options.page_load_strategy = 'normal'
    chrome_options.add_argument('--headless') # ensure GUI is off
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("start-maximized")

    # Setup WebDriver
    driver2 = uc.Chrome(chrome_options)

    storeID = []
    ln1 = []
    ln2 = []
    ln3 = []

    for val in missing_vals:
        driver2.get(loc)
        driver2.refresh()
        time.sleep(5)
        soup2 = BeautifulSoup(driver2.page_source, "html.parser")
        results2 = soup2.find("body")

        #get store ID
        if web_data['storeID'][val] == None:
            web_data['storeID'][val] = results2.get("class")[-1]

        #get address elements
        addr1 = results2.find("div", class_="address__line__1")
        addr2 = results2.find("div", class_="address__line__2")
        addr3 = results2.find("div", class_="address__line__3")

        if web_data['addressLine1'][val] == None:
            web_data['addressLine1'][val] = (addr1.get_text() if addr1 else None)
        if web_data['addressLine2'][val] == None:
            web_data['addressLine2'][val] = (addr2.get_text() if addr2 else None)
        if web_data['addressLine3'][val] == None:
            web_data['addressLine3'][val] = (addr3.get_text() if addr3 else None)

could not detect version_main.therefore, we are assuming it is chrome 108 or higher


In [16]:
web_data.isnull().sum()

locationName     0
phoneNumber      0
storeID          0
addressLine1     0
addressLine2    42
addressLine3     0
dtype: int64

In [17]:
web_data['storeID'] = web_data['storeID'].apply(lambda x: "".join(re.findall("[0-9]", x)))

In [18]:
def get_streetAddress(ln1, ln2):
    if ln2 != None:
        return ln1 + ln2
    else:
        return ln1

In [19]:
web_data['streetAddress'] = web_data.apply(lambda x: get_streetAddress(x.addressLine1, x.addressLine2), axis=1)

In [20]:
web_data['city'] = web_data['addressLine3'].apply(lambda x: x.split(', ')[0])
web_data['state'] = web_data['addressLine3'].apply(lambda x: "".join(re.findall("[a-zA-Z]", x.split(', ')[1])))
web_data['postalCode'] = web_data['addressLine3'].apply(lambda x: "".join(re.findall("[0-9]", x.split(', ')[1])))

In [21]:
web_data.head()

,locationName,phoneNumber,storeID,addressLine1,addressLine2,addressLine3,streetAddress,city,state,postalCode
0,San Jacinto,214-220-3911,1733,2100 Ross Avenue,Suite 120,"Dallas, TX 75201",2100 Ross AvenueSuite 120,Dallas,TX,75201
1,Park District - Petite Market & Bakery,972-913-2280,4691,2100 Olive Street,Suite 150,"Dallas, TX 75201",2100 Olive StreetSuite 150,Dallas,TX,75201
2,Dallas Love Field,214-358-7614,1685,8008 Cedar Springs Rd.,Terminal 2 Gate 6,"Dallas, TX 75235",8008 Cedar Springs Rd.Terminal 2 Gate 6,Dallas,TX,75235
3,Mockingbird,214-696-0800,1713,3072 Mockingbird Lane,None,"Dallas, TX 75205",3072 Mockingbird Lane,Dallas,TX,75205
4,Berkshire,945-279-0912,6206,6109 Berkshire Ln,Ste A,"Dallas, Texas 75225",6109 Berkshire LnSte A,Dallas,Texas,75225


In [22]:
# checking the data for null values
web_data.isnull().sum()

locationName      0
phoneNumber       0
storeID           0
addressLine1      0
addressLine2     42
addressLine3      0
streetAddress     0
city              0
state             0
postalCode        0
dtype: int64

In [23]:
final_web = web_data[['locationName','streetAddress','city','state','postalCode','phoneNumber','storeID']]
final_web.head()

,locationName,streetAddress,city,state,postalCode,phoneNumber,storeID
0,San Jacinto,2100 Ross AvenueSuite 120,Dallas,TX,75201,214-220-3911,1733
1,Park District - Petite Market & Bakery,2100 Olive StreetSuite 150,Dallas,TX,75201,972-913-2280,4691
2,Dallas Love Field,8008 Cedar Springs Rd.Terminal 2 Gate 6,Dallas,TX,75235,214-358-7614,1685
3,Mockingbird,3072 Mockingbird Lane,Dallas,TX,75205,214-696-0800,1713
4,Berkshire,6109 Berkshire LnSte A,Dallas,Texas,75225,945-279-0912,6206


In [24]:
final_web.drop_duplicates(inplace=True)

C:\Users\yasom\AppData\Local\Temp\ipykernel_46104\571506838.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_web.drop_duplicates(inplace=True)


In [25]:
# saving the scraped data
final_web.to_csv(r'final_web.csv', index=False)

In [26]:
# quitting the selenium session
driver.quit()
driver2.quit()

### Merging the data

In [27]:
# reading the data from drive
final_web = pd.read_csv(r'final_web.csv')
final_web.head()

,locationName,streetAddress,city,state,postalCode,phoneNumber,storeID
0,San Jacinto,2100 Ross AvenueSuite 120,Dallas,TX,75201,214-220-3911,1733
1,Park District - Petite Market & Bakery,2100 Olive StreetSuite 150,Dallas,TX,75201,972-913-2280,4691
2,Dallas Love Field,8008 Cedar Springs Rd.Terminal 2 Gate 6,Dallas,TX,75235,214-358-7614,1685
3,Mockingbird,3072 Mockingbird Lane,Dallas,TX,75205,214-696-0800,1713
4,Berkshire,6109 Berkshire LnSte A,Dallas,Texas,75225,945-279-0912,6206


In [28]:
# store ID present in reviews but not scraped data
rev_left = list(set(reviews['storeID']).difference(set(final_web['storeID'])))

In [29]:
reviews[reviews['storeID'].isin(rev_left)].drop_duplicates(subset='storeID')

,name,storeID,fullAddress,phone,website,overallRating,numberReviews,reviewer,reviewRating,reviewText,reviewDate
533,la Madeleine,2301,"2423 Katy Fwy Suite E, Houston, TX 77007",(832) 916-4001,https://lamadeleine.com/locations/houston-sawy...,4.1,484,D'Andra Walker,3,Every single time I PAY for extra dressing wit...,2024-03-13T19:11:29.322Z
14771,la Madeleine,1708,"6430 N MacArthur Blvd, Irving, TX 75039",(469) 385-1700,https://lamadeleine.com/locations/irving-macar...,4.2,1719,Shannon Hunt,5,I'm here at least 4-5 days a week. The staff i...,2024-03-18T19:11:48.823Z
15941,la Madeleine,1694,"2500 Cross Timbers Rd Suite 150 Ste 150, Flowe...",(469) 240-5888,https://lamadeleine.com/locations/flower-mound,4.0,626,Suzanne Beaudry,5,Very cool inside....wonderful atmosphere...Bea...,2024-03-18T19:12:16.417Z
22874,la Madeleine,1673,"5861 Crossroads Center Way, Bailey's Crossroad...",(703) 379-5551,https://lamadeleine.com/locations/baileys-cros...,4.1,862,Charles W,5,"Amazing staff, great food. French music was pr...",2024-03-17T19:12:01.938Z
23822,la Madeleine,1669,"810 W McDermott Dr, Allen, TX 75013",(972) 338-4927,https://lamadeleine.com/locations/allen,4.1,942,Alex Gutierrez,5,Food: 5/5 | Service: 5/5 | Atmosphere: 5/5,2024-03-12T19:12:16.348Z
39219,la Madeleine,1676,"6410 Camp Bowie Blvd, Fort Worth, TX 76116",(817) 654-0471,https://lamadeleine.com/locations/fort-worth-c...,4.3,798,Anita Lloyd (JewelITF),5,"It's a little pricy for me, but the food was g...",2024-03-12T19:15:10.955Z
51303,la Madeleine,1682,"520 W 15th St, Plano, TX 75075",(972) 398-3002,https://lamadeleine.com/locations/plano-collin...,4.3,1207,Tam Knox,4,Food: 5/5 | Service: 5/5 | Atmosphere: 5/5...,2024-03-17T19:14:49.846Z


Upon inspecting these values which do not match form the scraped data and the given reviews dataset, there is a discrepancy in the web page. I encountered 404 errors and pages with different id's. Therefore, I will drop these rows for further analysis.

In [30]:
reviews[reviews['storeID'].isin(rev_left)].drop_duplicates(subset='storeID')

,name,storeID,fullAddress,phone,website,overallRating,numberReviews,reviewer,reviewRating,reviewText,reviewDate
533,la Madeleine,2301,"2423 Katy Fwy Suite E, Houston, TX 77007",(832) 916-4001,https://lamadeleine.com/locations/houston-sawy...,4.1,484,D'Andra Walker,3,Every single time I PAY for extra dressing wit...,2024-03-13T19:11:29.322Z
14771,la Madeleine,1708,"6430 N MacArthur Blvd, Irving, TX 75039",(469) 385-1700,https://lamadeleine.com/locations/irving-macar...,4.2,1719,Shannon Hunt,5,I'm here at least 4-5 days a week. The staff i...,2024-03-18T19:11:48.823Z
15941,la Madeleine,1694,"2500 Cross Timbers Rd Suite 150 Ste 150, Flowe...",(469) 240-5888,https://lamadeleine.com/locations/flower-mound,4.0,626,Suzanne Beaudry,5,Very cool inside....wonderful atmosphere...Bea...,2024-03-18T19:12:16.417Z
22874,la Madeleine,1673,"5861 Crossroads Center Way, Bailey's Crossroad...",(703) 379-5551,https://lamadeleine.com/locations/baileys-cros...,4.1,862,Charles W,5,"Amazing staff, great food. French music was pr...",2024-03-17T19:12:01.938Z
23822,la Madeleine,1669,"810 W McDermott Dr, Allen, TX 75013",(972) 338-4927,https://lamadeleine.com/locations/allen,4.1,942,Alex Gutierrez,5,Food: 5/5 | Service: 5/5 | Atmosphere: 5/5,2024-03-12T19:12:16.348Z
39219,la Madeleine,1676,"6410 Camp Bowie Blvd, Fort Worth, TX 76116",(817) 654-0471,https://lamadeleine.com/locations/fort-worth-c...,4.3,798,Anita Lloyd (JewelITF),5,"It's a little pricy for me, but the food was g...",2024-03-12T19:15:10.955Z
51303,la Madeleine,1682,"520 W 15th St, Plano, TX 75075",(972) 398-3002,https://lamadeleine.com/locations/plano-collin...,4.3,1207,Tam Knox,4,Food: 5/5 | Service: 5/5 | Atmosphere: 5/5...,2024-03-17T19:14:49.846Z


In [31]:
id_merged = pd.merge(reviews, final_web[['storeID','locationName']], on='storeID', how='inner')

print(len(id_merged))

60885


In [32]:
id_merged

,name,storeID,fullAddress,phone,website,overallRating,numberReviews,reviewer,reviewRating,reviewText,reviewDate,locationName
0,la Madeleine,6206,"6109 Berkshire Ln Ste A, Dallas, TX 75225",(945) 279-0912,https://lamadeleine.com/locations/berkshire,3.1,12,Olga Garcia,2,How do you have 3 people working a lunch shift...,2024-03-05T19:11:20.386Z,Berkshire
1,la Madeleine,6206,"6109 Berkshire Ln Ste A, Dallas, TX 75225",(945) 279-0912,https://lamadeleine.com/locations/berkshire,3.1,12,Melissa Trevino,1,Just be aware of being double charged during t...,2024-03-05T19:11:20.386Z,Berkshire
2,la Madeleine,6206,"6109 Berkshire Ln Ste A, Dallas, TX 75225",(945) 279-0912,https://lamadeleine.com/locations/berkshire,3.1,12,Roger Stierman,5,"Always delicious,new crew being trainedFood: 5...",2024-02-27T19:11:20.386Z,Berkshire
3,la Madeleine,6206,"6109 Berkshire Ln Ste A, Dallas, TX 75225",(945) 279-0912,https://lamadeleine.com/locations/berkshire,3.1,12,Eric Brown,5,Very clean and nice location with friendly peo...,2024-02-27T19:11:20.387Z,Berkshire
4,la Madeleine,6206,"6109 Berkshire Ln Ste A, Dallas, TX 75225",(945) 279-0912,https://lamadeleine.com/locations/berkshire,3.1,12,AJ D.,2,It’s not even busy so I can’t BELIEVE how long...,2024-02-19T00:00:00.000Z,Berkshire
...,...,...,...,...,...,...,...,...,...,...,...,...
60880,la Madeleine,1737,"2207-I Kaliste Saloom Rd, Lafayette, LA 70508",(337) 504-7369,https://lamadeleine.com/locations/lafayette-sh...,4.4,547,Sandra Espinoza,5,Very pleased to visit this French Cafe. The br...,2016-03-19T00:00:00.000Z,Shops at Martial
60881,la Madeleine,1737,"2207-I Kaliste Saloom Rd, Lafayette, LA 70508",(337) 504-7369,https://lamadeleine.com/locations/lafayette-sh...,4.4,547,Natalie Rondeno,1,My sister and I had lunch there and we were in...,2015-03-19T00:00:00.000Z,Shops at Martial
60882,la Madeleine,1737,"2207-I Kaliste Saloom Rd, Lafayette, LA 70508",(337) 504-7369,https://lamadeleine.com/locations/lafayette-sh...,4.4,547,Bernadette Bamburg,5,NaN,2015-03-19T00:00:00.000Z,Shops at Martial
60883,la Madeleine,1737,"2207-I Kaliste Saloom Rd, Lafayette, LA 70508",(337) 504-7369,https://lamadeleine.com/locations/lafayette-sh...,4.4,547,Amanda Bourgeois,5,"Amazing atmosphere, amazing people, and most ...",2015-03-19T00:00:00.000Z,Shops at Martial


In [33]:
len(id_merged['storeID'].unique())

83

The final dataset has 83 unique locations after merging with the locations data

In [35]:
# savig the merged data
id_merged.to_csv(r'combined_data.csv', index=False)